In [31]:
import csv

class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                if isinstance(col, str) and len(col) > 50:
                    col = col[:50]
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
    
    
class CursorIter():
    def __init__(self, cursor):
        self.cursor = cursor
        self.field_names = [x[0] for x in self.cursor.description]
    
    def iterate(self):
        while True:
            row = self.cursor.fetchone()
            if row is None:
                break
            yield row
            
    @staticmethod    
    def from_query(conn, query):
        with conn.cursor() as cursor:
            cursor.execute(query)
            return CursorIter(cursor)
            
class CursorTable():
    def __init__(self, cursor):
        cursor_iter = CursorIter(cursor)
        self.table = ListTable()
        self.table.append(cursor_iter.field_names)
        for row in cursor_iter.iterate():
            self.table.append(row)
    
    def append_cursor(self, conn, query):
        with conn.cursor() as cursor:
            cursor.execute(query)
            for row in CursorIter(cursor).iterate():
                self.table.append(row)
            
    @staticmethod
    def from_query(conn, query):
        with conn.cursor() as cursor:
            cursor.execute(query)
            t = CursorTable(cursor)
        return t
            
    def _repr_html_(self):
        return self.table._repr_html_()
    
    def write_csv(self, file_name):
        with open(file_name, 'w') as f:
            writer = csv.writer(f)
            for row in self.table:
                writer.writerow(row)
#     def get_table(self):
#         table = ListTable()
#         table.append(self.cursor_iter.field_names)
#         for row in self.cursor_iter.iterate():
#             table.append(row)
#         return table

def geo_url(geo_id):
    geo_root_tpm = "http://ftp.ncbi.nlm.nih.gov/geo/{entity}/{entity_id}{prefix}nnn/{geo_id}/"
    entity_id = geo_id[:3]
    number = geo_id[3:]
    prefix = number[:-3]
    
    if entity_id == 'GSE':
        entity = 'series'
    elif entity_id == 'GSM':
        entity = 'samples'
    elif entity_id == 'GPL':
        entity = 'platforms'
    
    return geo_root_tpm.format(
        entity=entity,
        entity_id=entity_id,
        prefix=prefix,
        geo_id=geo_id
    )
    
    
    
    
    

In [4]:
import pymysql
conn = pymysql.connect(host='localhost', port=3306, user='scraper', passwd='scraper123', db='scraper')

In [5]:
platforms = ['GPL1708']
# tissues = ['Adipose', 'Artery', 'Brain', 'Lymphocytes', 'Leukemia', 'Fibroblasts', 
#            'Heart', 'Lung', 'Skeletal%Muscle', 'Nerve', 'Ski', 'Thyroid', 'Whole blood']

tissues = ['Adipose', 'Artery', 'Brain', 'Lymphocytes', 'Leukemia', 'Fibroblasts', 
           'Heart', 'Lung', 'Nerve', 'Ski', 'Thyroid', 'Whole blood']

with conn.cursor() as cursor:
    cursor.execute('select id from platform where accession in ({})'.format(
            ','.join(["'"+p+"'" for p in platforms])
    ))
    platform_ids = [r[0] for r in cursor.fetchall()]

In [3]:
from IPython.display import display

    
query_tpl = """
SELECT 
    sm.accession as accession,	
    se.accession as series,	
    sm.platform as platform,	
    '{tissue}' as tissue,	
    sm.title as title,	
    se.title as series_title,	
    sm.characteristics as characteristics,	
    sm.source_name as source_name,	
    se.overall_design as series_overall_design,	
    sm.sample_type as sample_type,	
    sm.organism as organism,	
    sm.extracted_molecule as extracted_molecule
FROM sample sm
JOIN sampleseries ss ON sm.id = ss.sample_ref_id
JOIN series se ON se.id = ss.series_ref_id
WHERE sm.platform_ref_id = {platform_id}
AND sm.source_name like '%{tissue}%'
{and_control}

LIMIT {limit}
"""

and_control = """
AND ((sm.title like '%control%' or sm.title like '%normal%')
    OR (sm.characteristics like '%control%' or sm.characteristics like '%normal%')
    OR (sm.source_name like '%control%' or sm.source_name like '%normal%'))
"""
t = None
for platform_id in platform_ids:
    for tissue in tissues:
        query = query_tpl.format(
            tissue=tissue,
            platform_id=platform_id,
            limit=30,
            and_control=and_control
        )
        if t is None:
            t = CursorTable.from_query(conn, query)
        else:
            t.append_cursor(conn, query)
# display(t)
t.write_csv('/Users/nikitaprianichnikov/Data/nm/request.csv')


NameError: name 'conn' is not defined

In [23]:
platform_id = platform_ids[0]
query_tpl = """
SELECT 
    se.accession
FROM sample sm
JOIN sampleseries ss ON sm.id = ss.sample_ref_id
JOIN series se ON se.id = ss.series_ref_id
WHERE sm.platform_ref_id = {platform_id}
GROUP BY se.accession
"""

# CursorTable.from_query(conn, query_tpl.format(platform_id=platform_id)).write_csv('/Users/nikitaprianichnikov/Data/nm/tmp/series.csv')
series = [r[0] for r in CursorIter.from_query(conn, query_tpl.format(platform_id=platform_id)).iterate()]

In [41]:
from sh import wget
from os import makedirs

out_dir = '/Users/nikitaprianichnikov/Data/nm/tmp/GPL1708-series/'
makedirs(out_dir, exist_ok=True)

for serie in series:
    print(serie)
    series_matrix_url = geo_url(serie)+'/matrix/'
    wget(series_matrix_url, 
         directory_prefix=out_dir,
         mirror=True,
         convert_links=True, 
#          adjust_extension=True, 
#          page_requisites=True,
         no_parent=True)
    



GSE10195


KeyboardInterrupt: 